In [48]:
import numpy as np
import pandas as pd

def getDF(pc, bl, seqs):
    parent_kids = pd.read_csv(pc, delimiter =',', names = ['Parent', 'Child']) # load up the parent child table
    branches = pd.DataFrame({"Lengths": list(open(bl))[0].split(',')}).astype(float) # load up the branch lengths
    combined = pd.concat([parent_kids, branches], axis = 1) # combine parent child table + branch lengths 

    n = min(parent_kids.loc[:,'Parent']) # smallest parent
    l = range(n-1,0, -1) #list of numbers from the min-1 to 1

    combined = pd.concat([combined, pd.DataFrame({"Sequences" : ["NA"]*len(combined["Parent"])})], axis = 1) # add NA for sequences

    sequences = pd.read_csv(seqs, header = None, delimiter = " ") # load up sequences for terminal nodes
    #sequences = sequences.iloc[::-1] # order from highest to lowest
    sequences.columns =["Child", "Sequence"] # give column names
    sequences["Sequence"] = [list(x) for x in sequences["Sequence"]]
    seql = len(sequences.iloc[1,1]) # sequence length
    
    merged_combined = pd.merge(combined, sequences, on='Child', how='left')
    combined['Sequences'] = merged_combined['Sequence']

    # combine sequences, branch lengths for terminal nodes
    #kids = pd.DataFrame({"Parent" : list(l), "Child" : ["NA"]*len(l), "Lengths" : [0]*len(l), "Sequences" : sequences["Lengths"]})

    # return whole combined df
    #return pd.concat([combined, kids], ignore_index = True, axis = 0), seql
    return combined, seql

tree, seql = getDF("/Users/lou89/Documents/Unil/Semestre1/Python/table.dat", "/Users/lou89/Documents/Unil/Semestre1/Python/branchlength.dat", "/Users/lou89/Documents/Unil/Semestre1/Python/msa.dat")
print(tree)
#print(seql)

m = max(tree.loc[:,'Parent']) # biggest parent


nts = ['A', 'C', 'G', 'T']

def get_vectors(i, m, nts): #i index of nucleotide, m biggest parent, nts nucleotides sequences
    lh_table = pd.DataFrame({"Node" : list(range(1,m+1,1)), "Vector" : m*[[0,0,0,0]], "Likelihood" : m*["NA"]})
    print(lh_table)
    c = [x for x in tree["Sequences"] if type(x) == list] #store sequences != NaN
    print(c)
    
    lh_table["Vector"][0] = [1,0,0,0]
    #print(lh_table["Vector"])
    for y in range(len(c)):
        #for x in range(len(nts)):
            #print(c[y][i])
            #print(nts[x])
            if c[y][i] == nts[0]:
                lh_table["Vector"][y] = [1,0,0,0]
                #lh_table["Vector"][y][x] = 1
                #lh_table.at[y, "Vector"][x] = 1
                #print(lh_table["Vector"][y])
                #print(lh_table["Vector"])
                #break
            elif c[y][i] == nts[1]:
                 lh_table["Vector"][y] = [0,1,0,0]
            
            elif c[y][i] == nts[2]:
                 lh_table["Vector"][y] = [0,0,1,0]

            else:
                lh_table["Vector"][y] = [0,0,0,1]
                #lh_table.at[y, "Vector"][x] = 0
                #print(lh_table["Vector"][y])
                #print(lh_table["Vector"])
                #continue
    #print(lh_table["Vector"][0][1])            
    return lh_table
print(get_vectors(0, m, nts))

'''
def get_lh(mu, tree, seql):
    subtree = tree.tail(list(tree["Child"]).count("NA"))
    #print(subtree)
    for x in subtree["Sequences"]:
        c = x[0]*4
        a = [x for x in c if c]
        print(a)

    #seql = len(subtree["Sequences"])
    #print(seql)
    
    #for x in subtree:
    '''

#print(tree["Sequences"][1][5])
#get_lh(0.3, tree, seql)


   Parent  Child  Lengths                                          Sequences
0       9      1     0.10  [A, G, A, T, C, A, A, G, A, T, C, A, A, G, A, ...
1       9      2     0.40  [A, G, C, T, C, A, A, G, C, T, C, A, A, G, C, ...
2       8      9     0.01                                                NaN
3       8      3     0.04  [C, G, C, T, A, T, C, G, C, T, A, T, C, G, C, ...
4       7      4     0.20  [C, G, T, T, A, C, C, G, T, T, A, C, C, G, T, ...
5       7      5     0.08  [C, G, C, T, A, C, C, G, C, T, A, C, C, G, C, ...
6       6      7     0.12                                                NaN
7       6      8     0.14                                                NaN
   Node        Vector Likelihood
0     1  [0, 0, 0, 0]         NA
1     2  [0, 0, 0, 0]         NA
2     3  [0, 0, 0, 0]         NA
3     4  [0, 0, 0, 0]         NA
4     5  [0, 0, 0, 0]         NA
5     6  [0, 0, 0, 0]         NA
6     7  [0, 0, 0, 0]         NA
7     8  [0, 0, 0, 0]         NA
8     9  [

C:\Users\lou89\AppData\Local\Temp\ipykernel_29604\605383009.py:45: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  lh_table["Vector"][0] = [1,0,0,0]
C:\Users\lou89\AppData\Local\Temp\ipykernel_29604\605383009.py:45: SettingWithCopyWarning: 
A 

'\ndef get_lh(mu, tree, seql):\n    subtree = tree.tail(list(tree["Child"]).count("NA"))\n    #print(subtree)\n    for x in subtree["Sequences"]:\n        c = x[0]*4\n        a = [x for x in c if c]\n        print(a)\n\n    #seql = len(subtree["Sequences"])\n    #print(seql)\n    \n    #for x in subtree:\n    '